In [1]:
from malt import MaltParser # source code from nltk library

In [3]:
#DO NOT COLLAPS THE PREPOSITION RELATIONSHIP
import pandas as pd
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk
from tqdm import tqdm
import json
#from datasets import load_dataset
from gensim import utils
from collections import defaultdict

import re

from word import Word
# from malt.malt import MaltParser # source code from nltk library
from malt import MaltParser # source code from nltk library

def process_dataset_removeStop_dontCollaps(docs, stop_words, pre_words, malt_parser_version='../maltparser-1.7.2', model_version='engmalt.linear-1.7.mco'):
    # initalize malt parser model
    mp = MaltParser(malt_parser_version, model_version, tagger=nltk.pos_tag)

    #Add more preprocessing:
    # replace single smart quote with single straight quote, so as to catch stopword contractions
    docs = [re.sub("[\u2018\u2019]", "'", doc) for doc in docs] #replace qoute with regualar qoutations
    #it removes the digits
    # docs = [re.sub('\d+', '', doc) for doc in docs] 
    docs = [re.sub('(\/.*?\.[\w:]+)', '', doc) for doc in docs]
    docs = [re.sub(r"http\S+", '', doc) for doc in docs]
    
    # create <doc_idx, tokenized_sent> list of sents
    sents = [
        (i, nltk.word_tokenize(sent))
        for i, doc in enumerate(docs)
        for sent in nltk.sent_tokenize(utils.to_unicode(str(doc).lower())) # convert doc to lowercase, and sentence tokenized.
    ]

    # unzip list of tuples
    doc_idxs, sents = zip(*sents)

    # create parser <generator> and loop through parser to produce dependency tree for each sentence
    parser = mp.parse_sents(sents, verbose=True)

    # define valid word
    stop_words.append('amp');stop_words.append('&amp');stop_words.append('&amp;')
    valid_word = lambda word: not word in stop_words and word.isalpha() and len(word) > 2

    # initalize dictionary for json output
    docs_dict = {
        'documents': dict((doc_idx, {'words': [], 'relns': [], 'arg2':[] , 'originaltext': []}) for doc_idx in doc_idxs),
        #'documents': dict((doc_idx, {'words': [], 'relns': []}) for doc_idx in doc_idxs),
        'vocab': [], 
        'vocab_relns': [],
        'vocab_arg':[],
    }
    # initalize vocab variables as sets (no duplicates)
    vocab = set()
    vocab_relns = set()
    vocab_arg = set()
    vocab_arg_test, vocab_relns_test = set(), set()

    i = 0
    # loop through list iterators
    for list_it in parser:
        tree = next(list_it)
        # check if valid tree, if not skip
        try:
            nodes = tree.nodes
        except:
            continue
        #print(nodes)
#         print('******000000*****')
        word_relns_hash = defaultdict(list)
        word_args_hash = defaultdict(list)
        for word_idx in nodes:
            if word_idx == 0: # skip first
                continue
#             if word_idx >= 13:
#                 pass
#             if nodes[word_idx]['word']=='persons.for':
#                 print('bad documents 1 : ', nodes[word_idx])
#                 return nodes[word_idx]
            deps = nodes[word_idx]['deps']
            
#             if (nodes[word_idx]['word'] in stop_words):
#                 continue
            # check for valid dependency relation
            if deps:
                for reln, idxs in deps.items():
                    for idx in idxs:
#                         if reln =='prep':
#                             #what if it has more than one deps? do function/stop words appear with more than one deps?@ZHILA I also need to check what if this prepositional term refers to several words what do do with that
# #                             if valid_word(nodes[idx]['word']):
#                             if nodes[idx]['word'] in prep_words:
#                                 reln = 'prep'+'.'+nodes[idx]['word']
#                                 try:
#                                     if len(list(nodes[idx]['deps'].values()))==0:
#                                         continue
#                                     elif type(list(nodes[idx]['deps'].values())[0])==list:
#                                         idx = list(nodes[idx]['deps'].values())[0][0]
#                                         if (not valid_word(nodes[word_idx]['word']) or (not valid_word(nodes[idx]['word']))):
#                                             continue
#                                     else:
#                                         idx = list(nodes[idx]['deps'].values())[0]
#                                         if (not valid_word(nodes[word_idx]['word']) or (not valid_word(nodes[idx]['word']))):
#                                             continue
#                                 except:
#                                     print('Error')
#                                     return (nodes, idx)
                            
#                             else:
#                                 continue
                            
                        
                        if (not valid_word(nodes[word_idx]['word']) or (not valid_word(nodes[idx]['word']))):
                            continue

                        dep_reln, gov_reln = f"{reln}.dep", f"{reln}.gov"
                        if gov_reln =="punct.gov":
                            continue
                        
                        
                        # add reln to word in hashmap
                        word_relns_hash[idx].append(dep_reln) # append to dep word
                        word_relns_hash[word_idx].append(gov_reln) # append to current word
#                         if nodes[word_idx]['word'] == 'and' or nodes[idx]['word'] == 'and':
#                             print('HT@')
                        word_args_hash[idx].append(nodes[word_idx]['word'])
                        word_args_hash[word_idx].append(nodes[idx]['word'])
                        #vocab_arg.add(nodes[idx]['word'])
                        #vocab_arg.add(nodes[word_idx]['word'])
                        #vocab_relns.add(dep_reln)
                        #vocab_relns.add(gov_reln)
                        
                        
        
        # check for valid hashmap
        if word_relns_hash:
            doc_idx = doc_idxs[i]

            # loop through hashmap items and append to dict for future storing
            for word_idx, relns in word_relns_hash.items():
                word = nodes[word_idx]['word']
                #relns = [reln for reln in relns if reln != "punct.gov"]
#                 if word =='persons.for':
#                     print(' some error in bad document: ', doc_idx)
#                     return doc_idx
                if valid_word(word):
                    vocab.add(word)
                    docs_dict['documents'][doc_idx]['words'].append(word)
                    docs_dict['documents'][doc_idx]['relns'].append(relns)
                    docs_dict['documents'][doc_idx]['originaltext'] = [docs[doc_idx]]
                    docs_dict['documents'][doc_idx]['arg2'].append(word_args_hash[word_idx])
                    try:
                        vocab_arg.update(word_args_hash[word_idx])
                        vocab_relns.update(relns)
                    except:
                        print(word_args_hash[word_idx])
#                         print(vocab_arg_test)
                        return 0
        
        i += 1

    docs_dict['vocab'] = list(vocab)
    docs_dict['vocab_relns'] = list(vocab_relns)
    docs_dict['vocab_arg'] = list(set(vocab_arg))
#     docs_dict['vocab_arg_test']=list(vocab_arg_test)
#     docs_dict['vocab_relns_test']= list(vocab_relns_test)

    return docs_dict

In [4]:
# set the stop words
import pickle
# with open("../Stopword_list",'rb') as read_file:
with open('extra_stopwords','rb') as read_file:
    more_stop_words = pickle.load(read_file)
# stop word initialization
# with open("data/utils/stopwords.txt") as f:
#     more_stop_words = f.read().splitlines()
stop_words = nltk.corpus.stopwords.words('english')
stop_words.extend(more_stop_words)

In [5]:
import pickle
with open('prep_words.txt', "r") as file:
    prep_words = [line.strip() for line in file]
# print(prep_words)

In [6]:
import mysql.connector

sql_db = mysql.connector.connect(host='127.0.0.1',user = 'covidAnalysis',password = 'k34p63MbDDcZ9yf4',
                                 database = 'covid19framing')
cursor = sql_db.cursor()

In [7]:
cursor.execute('show tables;')
results = cursor.fetchall()
# I changed the chosen column to be text_preproc and not text because of some issues in those rows wherein text_preproc is null and text exist...
cursor.execute("select * from articles where text_preproc is not NULL")
results = cursor.fetchall()
len(results)

3699

In [8]:
originaltext = [item[1].decode() for item in results] #originaltextoriginaltext[1]
corpus = pd.Series(originaltext)

In [9]:
sample_size = int(len(corpus) * .1)
sample_corpus = corpus.sample(n=sample_size)
len(sample_corpus)

369

In [10]:
docs_dict = process_dataset(sample_corpus, stop_words, prep_words)

-----------------------------------------------------------------------------
                          MaltParser 1.7.2                             
-----------------------------------------------------------------------------
         MALT (Models and Algorithms for Language Technology) Group          
             Vaxjo University and Uppsala University                         
                             Sweden                                          
-----------------------------------------------------------------------------

Started: Fri Aug 04 17:10:26 EDT 2023
  Transition system    : Projective
  Parser configuration : Stack
  Feature model        : eng-liblinear.xml
  Classifier           : liblinear
  Data Format          : /engmalt.linear-1.7/conllx.xml
.          	      1	      5s	    365MB
.          	     10	      5s	    376MB
.          	    100	      6s	    464MB
..........	   1000	     14s	    780MB
..........	   2000	     19s	    527MB
..........	   3000	     23s

In [44]:
# test_doc = 'I thank the medical volunteers who have come forward, and I ask for others who have not yet raised their hand to help to do so now – your skills and support continues to be needed.'
# d_test = pd.Series(test_doc)
# tst= process_dataset(d_test, stop_words, pre_words)

In [12]:
valid_word = lambda word: not word in stop_words and word.isalpha() and len(word) > 2

In [13]:
argupdates = docs_dict['vocab_arg']
len(argupdates)
'the' in argupdates, valid_word('the')

(False, False)

In [17]:
rels = docs_dict['vocab_relns']
len(rels)
'prep.for.dep' in rels

False

In [23]:
(not valid_word('and') or (not valid_word('or')))

True

In [18]:
json_object = json.dumps(docs_dict, indent=4)
with open("sample_10_without_stopwords.json", "w") as f:
    f.write(json_object)

In [13]:
# with open("sample_10.json") as json_file:
#     o = json.load(json_file)
# arg2 =o['vocab_arg']